In [7]:
import pandas as pd
from collections import defaultdict
import heapq
import tkinter as tk
from tkinter import ttk, messagebox
import random
import os
import math

In [8]:
class Graph:
    """
    Lớp biểu diễn đồ thị vô hướng, tích hợp sẵn các thuật toán cơ bản
    và thuật toán tìm đường đi ngắn nhất Dijkstra.
    """
    def __init__(self):
        """Khởi tạo một đồ thị rỗng."""
        self.adj_list = defaultdict(list)
        self.vertices = set()

    def add_edge(self, v_from, v_to, weight):
        """Thêm một cạnh vô hướng vào đồ thị."""
        try:
            # Đảm bảo trọng số là số thực để xử lý nhất quán
            weight = float(weight)
        except (ValueError, TypeError):
            print(f"Cảnh báo: Trọng số '{weight}' không hợp lệ cho cạnh ({v_from}, {v_to}). Bỏ qua.")
            return

        self.adj_list[v_from].append((v_to, weight))
        self.adj_list[v_to].append((v_from, weight))
        self.vertices.update([v_from, v_to])

    def get_neighbors(self, vertex):
        return self.adj_list.get(vertex, [])

    def num_vertices(self):
        return len(self.vertices)

    def has_negative_weight(self):
        return any(w < 0 for nbrs in self.adj_list.values() for _, w in nbrs)

    def load_from_csv(self, file_path):
        """
        Nạp đồ thị từ file CSV.
        Trả về True nếu nạp thành công, False nếu có lỗi.
        """
        try:
            df = pd.read_csv(file_path)
            if len(df.columns) == 3:
                df.columns = ['v_from', 'v_to', 'weight']
            else:
                print(f"Lỗi: File CSV '{file_path}' phải có đúng 3 cột.")
                return False 
                
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file '{file_path}'.")
            return False 
        except Exception as e:
            print(f"Lỗi khi đọc file CSV: {e}")
            return False 

        for _, row in df.iterrows():
            self.add_edge(row['v_from'], row['v_to'], row['weight'])
        print(f"Đã nạp {len(df)} cạnh từ file '{file_path}'.")
        return True 
    
    def show_adj_list(self):
        """Hiển thị danh sách kề của đồ thị."""
        if not self.adj_list:
            print("Đồ thị rỗng.")
            return
        print("Danh sách kề của đồ thị:")
        for v in sorted(self.adj_list.keys()):
            edges = ', '.join([f"{nbr}({w})" for nbr, w in self.adj_list[v]])
            print(f"  {v} -> {edges}")

    def dijkstra(self, start_vertex, end_vertex):
        """
        Thuật toán Dijkstra chuẩn, chạy một lần và trả về kết quả cuối cùng.
        """
        if start_vertex not in self.vertices or end_vertex not in self.vertices:
            print("Lỗi: Đỉnh bắt đầu hoặc kết thúc không tồn tại trong đồ thị.")
            return None, float('inf')
        if self.has_negative_weight():
            print("Cảnh báo: Đồ thị có trọng số âm, Dijkstra có thể cho kết quả không chính xác.")
        distances = {v: float('inf') for v in self.vertices}
        previous_nodes = {v: None for v in self.vertices}
        distances[start_vertex] = 0
        priority_queue = [(0, start_vertex)]
        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)
            if current_distance > distances[current_vertex]:
                continue
            if current_vertex == end_vertex:
                break
            for neighbor, weight in self.get_neighbors(current_vertex):
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (distance, neighbor))
        path = []
        current = end_vertex
        if distances[current] == float('inf'):
            return None, float('inf')
        while current is not None:
            path.insert(0, current)
            current = previous_nodes[current]
        return path, distances[end_vertex]

    def dijkstra_generator(self, start_vertex, end_vertex):
        """
        Phiên bản "generator" của thuật toán Dijkstra, trả về (yield) trạng thái
        của thuật toán ở mỗi bước để phục vụ cho việc mô phỏng.
        """
        if self.has_negative_weight():
            yield {
                'log_message': "Cảnh báo: Đồ thị có trọng số âm, Dijkstra có thể không chính xác.",
                'distances': {}, 'previous_nodes': {}, 'visited': set(), 'priority_queue_nodes': []
            }
        distances = {v: float('inf') for v in self.vertices}
        previous_nodes = {v: None for v in self.vertices}
        distances[start_vertex] = 0
        priority_queue = [(0, start_vertex)]
        visited = set()
        log_msg_1 = "--- BẮT ĐẦU THUẬT TOÁN DIJKSTRA ---"
        log_msg_2 = f"Khởi tạo: Đặt khoảng cách tới '{start_vertex}' là 0, các đỉnh khác là vô cực."
        log_msg_3 = f"Hàng đợi ưu tiên (PQ): {[(f'{d:.1f}', v) for d,v in priority_queue]}"
        yield {
            'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
            'visited': set(visited), 'priority_queue_nodes': [v for _, v in priority_queue],
            'current_vertex': None, 'relaxing_edge': (None, None),
            'log_message': f"{log_msg_1}\n{log_msg_2}\n{log_msg_3}"
        }
        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)
            pq_nodes = [v for _, v in priority_queue]
            log_msg_loop = f"\n{'='*40}\nVòng lặp mới: Lấy '{current_vertex}' (khoảng cách {current_distance:.1f}) ra khỏi PQ."
            log_msg_pq = f"PQ hiện tại: {[f'({d:.1f}, {v})' for d,v in sorted(priority_queue)]}"
            yield {
                'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                'visited': set(visited), 'priority_queue_nodes': list(pq_nodes),
                'current_vertex': current_vertex, 'relaxing_edge': (None, None),
                'log_message': f"{log_msg_loop}\n{log_msg_pq}"
            }
            if current_distance > distances[current_vertex]:
                log_msg_skip = f"Bỏ qua '{current_vertex}' vì đã có đường đi ngắn hơn ({distances[current_vertex]:.1f}) được tìm thấy."
                yield {
                    'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                    'visited': set(visited), 'priority_queue_nodes': list(pq_nodes),
                    'current_vertex': current_vertex, 'relaxing_edge': (None, None),
                    'log_message': log_msg_skip
                }
                continue
            if current_vertex in visited: continue
            visited.add(current_vertex)
            yield {
                'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                'visited': set(visited), 'priority_queue_nodes': list(pq_nodes),
                'current_vertex': current_vertex, 'relaxing_edge': (None, None),
                'log_message': f"Đánh dấu '{current_vertex}' là đã duyệt (visited)."
            }
            if current_vertex == end_vertex:
                log_msg_end = f"Đã đến đích '{end_vertex}'. Dừng thuật toán."
                yield {
                    'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                    'visited': set(visited), 'priority_queue_nodes': list(pq_nodes),
                    'current_vertex': current_vertex, 'relaxing_edge': (None, None),
                    'log_message': log_msg_end
                }
                break
            log_msg_neighbors = f"Duyệt các hàng xóm của '{current_vertex}':"
            yield {
                'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                'visited': set(visited), 'priority_queue_nodes': list(pq_nodes),
                'current_vertex': current_vertex, 'relaxing_edge': (None, None),
                'log_message': log_msg_neighbors
            }
            for neighbor, weight in self.get_neighbors(current_vertex):
                if neighbor in visited:
                    log_msg_n_skip = f"  - Bỏ qua '{neighbor}' vì đã được duyệt xong."
                    yield {
                        'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                        'visited': set(visited), 'priority_queue_nodes': list(pq_nodes),
                        'current_vertex': current_vertex, 'relaxing_edge': (current_vertex, neighbor),
                        'log_message': log_msg_n_skip
                    }
                    continue
                distance = current_distance + weight
                log_msg_n_1 = f"  - Xét cạnh ({current_vertex}, {neighbor}) với trọng số {weight}:"
                log_msg_n_2 = f"    Khoảng cách mới tới '{neighbor}' = {current_distance:.1f} + {weight} = {distance:.1f}"
                yield {
                    'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                    'visited': set(visited), 'priority_queue_nodes': list(pq_nodes),
                    'current_vertex': current_vertex, 'relaxing_edge': (current_vertex, neighbor),
                    'log_message': f"{log_msg_n_1}\n{log_msg_n_2}"
                }
                if distance < distances[neighbor]:
                    log_msg_update_1 = f"    => Cập nhật khoảng cách của '{neighbor}' từ {distances[neighbor]} thành {distance:.1f}."
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (distance, neighbor))
                    pq_nodes_new = [v for _, v in priority_queue]
                    log_msg_update_2 = f"    Đưa (dist:{distance:.1f}, node:'{neighbor}') vào PQ."
                    log_msg_update_3 = f"    PQ sau khi cập nhật: {[f'({d:.1f}, {v})' for d,v in sorted(priority_queue)]}"
                    yield {
                        'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                        'visited': set(visited), 'priority_queue_nodes': list(pq_nodes_new),
                        'current_vertex': current_vertex, 'relaxing_edge': (current_vertex, neighbor),
                        'log_message': f"{log_msg_update_1}\n{log_msg_update_2}\n{log_msg_update_3}"
                    }
                else:
                    log_msg_no_update = f"    => Giữ nguyên khoảng cách cũ của '{neighbor}' là {distances[neighbor]:.1f}."
                    yield {
                        'distances': dict(distances), 'previous_nodes': dict(previous_nodes),
                        'visited': set(visited), 'priority_queue_nodes': list(pq_nodes),
                        'current_vertex': current_vertex, 'relaxing_edge': (current_vertex, neighbor),
                        'log_message': log_msg_no_update
                    }
        
        path = []
        current = end_vertex
        if distances[current] == float('inf'):
            return {
                'start': start_vertex,
                'end': end_vertex,
                'final_path': None,
                'final_distance': float('inf'),
                'distances': dict(distances),
                'previous_nodes': dict(previous_nodes),
                'visited': set(visited)
            }
        while current is not None:
            path.insert(0, current)
            if len(path) > len(self.vertices):
                print("Lỗi: Vòng lặp vô hạn khi tái tạo đường đi.")
                path = [] 
                break
            current = previous_nodes.get(current)
            
        return {
            'start': start_vertex,
            'end': end_vertex,
            'final_path': path,
            'final_distance': distances[end_vertex],
            'distances': dict(distances),
            'previous_nodes': dict(previous_nodes),
            'visited': set(visited)
        }

In [ ]:
class DijkstraVisualizer(tk.Tk):
    def __init__(self, graph):
        super().__init__()
        self.graph = graph
        self.node_display_positions = {}
        self.node_objects = {}
        self.edge_objects = {}
        self.algorithm_generator = None
        self.running_continuously = False

        self._drag_data = {"node_name": None}

        self.root_pane = None
        self.sidebar_frame = None
        self.sidebar_tree = None
        self.sidebar_width = 350

        self.history = []
        self.history_final_state = None
        self.current_step_index = -1

        self.animation_speed_scale = None
        self.step_back_button = None
        self.step_forward_button = None
        self.toggle_sidebar_button = None

        self.title("Minh họa thuật toán Dijkstra")
        self.minsize(1200, 650)

        self._setup_ui()
        self._populate_selectors()

    def _setup_ui(self):
        # --- Thanh trên cùng (Top Bar) ---
        top_frame = ttk.Frame(self, padding="10")
        top_frame.pack(side="top", fill="x")

        self.toggle_sidebar_button = ttk.Button(top_frame, text="<<<", width=4, command=self._toggle_sidebar)
        self.toggle_sidebar_button.pack(side="left", padx=(0, 10))

        ttk.Label(top_frame, text="Bắt đầu:").pack(side="left", padx=(0, 5))
        self.start_node_combo = ttk.Combobox(top_frame, state="readonly", width=10)
        self.start_node_combo.pack(side="left", padx=(0, 10))

        ttk.Label(top_frame, text="Kết thúc:").pack(side="left", padx=(0, 5))
        self.end_node_combo = ttk.Combobox(top_frame, state="readonly", width=10)
        self.end_node_combo.pack(side="left", padx=(0, 10))

        self.find_path_button = ttk.Button(top_frame, text="Tìm đường đi", command=self._start_algorithm)
        self.find_path_button.pack(side="left", padx=5)

        self.step_back_button = ttk.Button(top_frame, text="< Step Back", command=self._step_back, state="disabled")
        self.step_back_button.pack(side="left", padx=(10, 5))

        self.step_forward_button = ttk.Button(top_frame, text="Step Forward >", command=self._step_forward, state="disabled")
        self.step_forward_button.pack(side="left", padx=5)

        self.run_button = ttk.Button(top_frame, text="Chạy liên tục", command=self._toggle_continuous_run, state="disabled")
        self.run_button.pack(side="left", padx=5)

        self.reset_button = ttk.Button(top_frame, text="Reset", command=self._reset_visualization)
        self.reset_button.pack(side="right", padx=10)

        self.regenerate_button = ttk.Button(top_frame, text="Sắp xếp lại", command=self._regenerate_layout)
        self.regenerate_button.pack(side="right", padx=0)

        self.animation_speed_scale = ttk.Scale(top_frame, from_=100, to=2000, orient="horizontal", length=150)
        self.animation_speed_scale.set(1500)
        self.animation_speed_scale.pack(side="right", padx=5)
        ttk.Label(top_frame, text="Tốc độ:").pack(side="right", padx=(10, 5))

        self.root_pane = ttk.PanedWindow(self, orient="horizontal")
        self.root_pane.pack(side="top", fill="both", expand=True, padx=10, pady=(0, 10))

        # --- Sidebar (Trái) ---
        self.sidebar_frame = ttk.Frame(self.root_pane, width=self.sidebar_width)
        self.sidebar_frame.pack_propagate(False)
        ttk.Label(self.sidebar_frame, text="Trạng thái Thuật toán", font=("-weight bold")).pack(pady=5)
        tree_frame = ttk.Frame(self.sidebar_frame)
        tree_frame.pack(fill="both", expand=True, padx=5, pady=5)
        cols = ("vertex", "known", "cost", "path", "note")
        self.sidebar_tree = ttk.Treeview(tree_frame, columns=cols, show="headings")
        self.sidebar_tree.heading("vertex", text="Vertex")
        self.sidebar_tree.column("vertex", width=60, anchor="center", stretch=False) 
        self.sidebar_tree.heading("known", text="Known")
        self.sidebar_tree.column("known", width=50, anchor="center", stretch=False) 
        self.sidebar_tree.heading("cost", text="Cost")
        self.sidebar_tree.column("cost", width=60, anchor="e", stretch=False)
        self.sidebar_tree.heading("path", text="Path")
        self.sidebar_tree.column("path", width=100, stretch=False)
        self.sidebar_tree.heading("note", text="Note")
        self.sidebar_tree.column("note", width=60, stretch=False)
        tree_scroll_y = ttk.Scrollbar(tree_frame, orient="vertical", command=self.sidebar_tree.yview)
        tree_scroll_x = ttk.Scrollbar(tree_frame, orient="horizontal", command=self.sidebar_tree.xview)
        self.sidebar_tree.configure(yscrollcommand=tree_scroll_y.set, xscrollcommand=tree_scroll_x.set)
        tree_scroll_y.pack(side="right", fill="y")
        tree_scroll_x.pack(side="bottom", fill="x")
        self.sidebar_tree.pack(fill="both", expand=True)
        self.root_pane.add(self.sidebar_frame, weight=0)

        
        right_pane = ttk.PanedWindow(self.root_pane, orient="vertical")
        canvas_frame = ttk.Frame(right_pane)
        h_scroll = ttk.Scrollbar(canvas_frame, orient='horizontal')
        v_scroll = ttk.Scrollbar(canvas_frame, orient='vertical')
        self.canvas = tk.Canvas(canvas_frame, bg="#fcfcfc", highlightthickness=0,
                                xscrollcommand=h_scroll.set,
                                yscrollcommand=v_scroll.set)
        h_scroll.config(command=self.canvas.xview)
        v_scroll.config(command=self.canvas.yview)
        canvas_frame.grid_rowconfigure(0, weight=1)
        canvas_frame.grid_columnconfigure(0, weight=1)
        self.canvas.grid(row=0, column=0, sticky='nsew')
        v_scroll.grid(row=0, column=1, sticky='ns')
        h_scroll.grid(row=1, column=0, sticky='ew')
        
        self.canvas.bind("<Configure>", self._initial_graph_layout)

        self.canvas.bind("<ButtonPress-1>", self._on_node_press)
        self.canvas.bind("<B1-Motion>", self._on_node_motion)
        self.canvas.bind("<ButtonRelease-1>", self._on_node_release)
        right_pane.add(canvas_frame, weight=3)

        bottom_frame = ttk.Frame(right_pane)
        bottom_frame.grid_rowconfigure(1, weight=1) 
        bottom_frame.grid_columnconfigure(0, weight=1) 

        ttk.Label(bottom_frame, text="Chi tiết các bước chạy (Terminal):").grid(
            row=0, column=0, sticky="w", padx=10, pady=(5,0)
        )

        text_frame = ttk.Frame(bottom_frame)
        text_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=(5, 10))
        self.terminal = tk.Text(text_frame, wrap="word", state="disabled", height=10,
                                bg="#2b2b2b", fg="white", font=("Consolas", 10))
        scrollbar = ttk.Scrollbar(text_frame, command=self.terminal.yview)
        self.terminal.config(yscrollcommand=scrollbar.set)
        scrollbar.pack(side="right", fill="y")
        self.terminal.pack(side="left", fill="both", expand=True)

        right_pane.add(bottom_frame, weight=1)

        self.root_pane.add(right_pane, weight=1)
        self.root_pane.sashpos(0, self.sidebar_width)

    def _initial_graph_layout(self, event):
        if self.node_display_positions:
            return

        width = event.width
        height = event.height
        
        # Tránh tính toán khi canvas chưa có kích thước thực tế
        if width < 50 or height < 50:
            return

        # Tính toán vị trí các đỉnh dựa trên kích thước thực tế của canvas
        self._calculate_node_positions_force_directed(width, height)
        
        # Vẽ đồ thị lần đầu tiên
        self._reset_visualization()


    def _log(self, message):
        self.terminal.config(state="normal")
        self.terminal.insert("end", message + "\n")
        self.terminal.see("end")
        self.terminal.config(state="disabled")

    def _populate_selectors(self):
        sorted_vertices = sorted(list(self.graph.vertices))
        self.start_node_combo['values'] = sorted_vertices
        self.end_node_combo['values'] = sorted_vertices
        if sorted_vertices:
            self.start_node_combo.current(0)
            self.end_node_combo.current(len(sorted_vertices) - 1)

    def _calculate_node_positions_force_directed(self, width, height):
        if not self.graph.vertices: return
        
        nodes = list(self.graph.vertices)
        num_nodes = len(nodes)
        
        # Đặt các đỉnh trong một khu vực có đệm để không bị sát viền
        padding = 80
        positions = {node: (random.uniform(padding, width-padding), random.uniform(padding, height-padding)) for node in nodes}

        area = width * height
        k = 0.9 * math.sqrt(area / num_nodes) 
        iterations = 25 
        temp = width / 10.0

        for i in range(iterations):
            disp = {node: [0.0, 0.0] for node in nodes}
            for u in nodes:
                for v in nodes:
                    if u != v:
                        dx = positions[u][0] - positions[v][0]
                        dy = positions[u][1] - positions[v][1]
                        dist = math.sqrt(dx**2 + dy**2)
                        if dist > 0:
                            force = k*k / dist
                            disp[u][0] += (dx / dist) * force
                            disp[u][1] += (dy / dist) * force
            
            for u in nodes:
                for v, _ in self.graph.get_neighbors(u):
                    if u not in positions or v not in positions: continue
                    dx = positions[v][0] - positions[u][0]
                    dy = positions[v][1] - positions[u][1]
                    dist = math.sqrt(dx**2 + dy**2)
                    if dist > 0:
                        force = (dist**2) / k
                        disp[u][0] += (dx / dist) * force
                        disp[u][1] += (dy / dist) * force
                        disp[v][0] -= (dx / dist) * force
                        disp[v][1] -= (dy / dist) * force

            for node in nodes:
                dx, dy = disp[node]
                dist = math.sqrt(dx**2 + dy**2)
                if dist > 0:
                    new_x = positions[node][0] + (dx / dist) * min(dist, temp)
                    new_y = positions[node][1] + (dy / dist) * min(dist, temp)
                    positions[node] = (
                        min(width - padding, max(padding, new_x)),
                        min(height - padding, max(padding, new_y))
                    )
            temp *= (1.0 - (i / iterations))

        self.node_display_positions = positions

    def _update_canvas_scrollregion(self):
        """Tính toán và đặt vùng có thể cuộn để bao trọn tất cả các đối tượng trên canvas."""
        # bbox("all") sẽ trả về một hình chữ nhật bao quanh tất cả các item trên canvas
        bbox = self.canvas.bbox("all")
        if bbox:
            padding = 50 
            self.canvas.config(scrollregion=(
                bbox[0] - padding,
                bbox[1] - padding,
                bbox[2] + padding,
                bbox[3] + padding
            ))
        else:
             self.canvas.config(scrollregion=(0, 0, 1, 1))

    def _get_text_position_and_anchor(self, node_name):
        if node_name not in self.node_display_positions:
            return 0, 0, "center"
        x, y = self.node_display_positions[node_name]
        node_radius = 20
        padding = 5
        return x, y - node_radius - padding, "s"


    def _draw_graph(self):
        self.canvas.delete("all")
        self.node_objects.clear()
        self.edge_objects.clear()
        if not self.node_display_positions: return

        for u in self.graph.adj_list:
            for v, w in self.graph.adj_list[u]:
                key = tuple(sorted((u, v)))
                if key not in self.edge_objects:
                    if u not in self.node_display_positions or v not in self.node_display_positions: continue
                    x1, y1 = self.node_display_positions[u]
                    x2, y2 = self.node_display_positions[v]
                    line = self.canvas.create_line(x1, y1, x2, y2, fill="gray", width=1.5, tags="edge")
                    weight_text = self.canvas.create_text((x1+x2)/2, (y1+y2)/2 - 8, text=str(w),
                                                fill="#555555", font=("Arial", 9, "italic"), tags="weight")
                    self.edge_objects[key] = {'line': line, 'weight': weight_text}

        node_radius = 20
        for node, (x, y) in self.node_display_positions.items():
            oval = self.canvas.create_oval(x-node_radius, y-node_radius, x+node_radius, y+node_radius,
                                           fill="lightblue", outline="black", width=1.5, tags=("node", node))
            
            text_x, text_y, anchor = self._get_text_position_and_anchor(node)
            text = self.canvas.create_text(text_x, text_y, text=node, anchor=anchor,
                                           font=("Arial", 10, "bold"), tags=("node_text", node))
            self.node_objects[node] = {'oval': oval, 'text': text}
        
        # Cập nhật vùng cuộn sau khi vẽ xong
        self._update_canvas_scrollregion()
        
    def _update_canvas(self, state):
        for key, obj in self.edge_objects.items():
            self.canvas.itemconfig(obj['line'], fill="gray", width=1.5)
        for node, obj in self.node_objects.items():
            color = "lightblue"
            if node in state.get('visited', set()): color = "#90ee90"
            if node in state.get('priority_queue_nodes', []): color = "#ffd700"
            if node == state.get('current_vertex'): color = "#ff6b6b"
            self.canvas.itemconfig(obj['oval'], fill=color)
        u, v = state.get('relaxing_edge', (None, None))
        if u and v:
            key = tuple(sorted((u, v)))
            if key in self.edge_objects:
                self.canvas.itemconfig(self.edge_objects[key]['line'], fill="red", width=2.5)
        path = state.get('final_path')
        if path:
            for node in path:
                if node in self.node_objects:
                    self.canvas.itemconfig(self.node_objects[node]['oval'], fill="#4169e1")
            for i in range(len(path) - 1):
                u, v = path[i], path[i+1]
                key = tuple(sorted((u, v)))
                if key in self.edge_objects:
                    self.canvas.itemconfig(self.edge_objects[key]['line'], fill="blue", width=4)
            if self.node_objects.get(path[0]):
                self.canvas.itemconfig(self.node_objects[path[0]]['oval'], fill="green")
            if self.node_objects.get(path[-1]):
                self.canvas.itemconfig(self.node_objects[path[-1]]['oval'], fill="blue")

    def _reset_visualization(self):
        self.running_continuously = False
        self.run_button.config(text="Chạy liên tục")
        self.algorithm_generator = None
        self.history = []
        self.history_final_state = None
        self.current_step_index = -1
        if self.node_display_positions:
            self._draw_graph()
        self.terminal.config(state="normal")
        self.terminal.delete("1.0", "end")
        self.terminal.config(state="disabled")
        if self.sidebar_tree:
            self.sidebar_tree.delete(*self.sidebar_tree.get_children())
            self._populate_sidebar_initial()
        self.find_path_button.config(state="normal")
        self.start_node_combo.config(state="readonly")
        self.end_node_combo.config(state="readonly")
        self.step_forward_button.config(state="disabled")
        self.step_back_button.config(state="disabled")
        self.run_button.config(state="disabled")

    def _regenerate_layout(self):
        """Tính toán lại vị trí các đỉnh một cách ngẫu nhiên và vẽ lại đồ thị."""
        # Lấy kích thước hiện tại của canvas
        width = self.canvas.winfo_width()
        height = self.canvas.winfo_height()

        # Đảm bảo canvas đã có kích thước để tránh lỗi
        if width < 50 or height < 50:
            self._log("Không thể sắp xếp lại: Cửa sổ quá nhỏ.")
            return
        self._log("--- Đang tạo lại layout cho đồ thị... ---")
        self._calculate_node_positions_force_directed(width, height)
        self._reset_visualization()

    def _start_algorithm(self):
        start_vertex = self.start_node_combo.get()
        end_vertex = self.end_node_combo.get()
        if not start_vertex or not end_vertex:
            messagebox.showwarning("Thiếu thông tin", "Vui lòng chọn cả điểm bắt đầu và kết thúc.")
            return
        
        # Chỉ reset trạng thái thuật toán, không vẽ lại đồ thị
        self.running_continuously = False
        self.run_button.config(text="Chạy liên tục")
        self.algorithm_generator = None
        self.history = []
        self.history_final_state = None
        self.current_step_index = -1
        # Vẽ lại đồ thị ở trạng thái ban đầu
        self._draw_graph()
        # Xóa terminal và sidebar
        self.terminal.config(state="normal")
        self.terminal.delete("1.0", "end")
        self.terminal.config(state="disabled")
        if self.sidebar_tree:
            self.sidebar_tree.delete(*self.sidebar_tree.get_children())
            self._populate_sidebar_initial()
        
        self.find_path_button.config(state="disabled")
        self.start_node_combo.config(state="disabled")
        self.end_node_combo.config(state="disabled")
        # Hàm chạy thuật toán từ lớp Graph
        self.algorithm_generator = self.graph.dijkstra_generator(start_vertex, end_vertex)
        self.history = []
        self.history_final_state = None
        self._log("Đang tính toán các bước của thuật toán...")
        try:
            while True:
                self.history.append(next(self.algorithm_generator))
        except StopIteration as e:
            self.history_final_state = e.value
        self._log(f"Đã tạo xong lịch sử với {len(self.history)} bước.")
        self.current_step_index = -1
        if self.history:
            self.step_forward_button.config(state="normal")
            self.step_back_button.config(state="disabled")
            self.run_button.config(state="normal")
            self._step_forward()
        else:
            self._log("Không có bước nào để thực hiện.")

    def _step_forward(self):
        if self.running_continuously:
            self._toggle_continuous_run() 
            return

        if self.current_step_index < len(self.history) - 1:
            self.current_step_index += 1
            state = self.history[self.current_step_index]
            self._update_ui_from_state(state)
            self.step_back_button.config(state="normal")
            if self.current_step_index == len(self.history) - 1:
                self.step_forward_button.config(state="disabled")
                self.run_button.config(state="disabled")
                self._handle_algorithm_end()

    def _step_back(self):
        if self.running_continuously:
            self._toggle_continuous_run()
        if self.current_step_index > 0:
            self.current_step_index -= 1
            state = self.history[self.current_step_index]
            self._update_ui_from_state(state)
            self._log(f"--- LÙI BƯỚC (Quay về trạng thái {self.current_step_index}) ---")
            self.step_forward_button.config(state="normal")
            self.run_button.config(state="normal")
            if self.current_step_index == 0:
                self.step_back_button.config(state="disabled")

    def _handle_algorithm_end(self):
        if not self.history_final_state: return
        final_state = self.history_final_state
        self._update_canvas(final_state)
        self._log("\n" + "--- KẾT THÚC THUẬT TOÁN ---")
        if final_state.get('final_path'):
            path_str = " -> ".join(final_state['final_path'])
            dist = final_state['final_distance']
            self._log(f"Đường đi ngắn nhất: {path_str}")
            self._log(f"Tổng khoảng cách: {dist}")
        else:
            self._log(f"Không tìm thấy đường đi từ {final_state['start']} đến {final_state['end']}.")

    def _toggle_continuous_run(self):
        if self.running_continuously:
            self.running_continuously = False
            self.run_button.config(text="Chạy liên tục")
            if self.current_step_index > 0:
                self.step_back_button.config(state="normal")
        else:
            if self.current_step_index >= len(self.history) - 1:
                return

            self.running_continuously = True
            self.run_button.config(text="Tạm dừng")
            self.step_back_button.config(state="disabled")
            self.step_forward_button.config(state="disabled")
            self._run_loop()

    def _run_loop(self):
        if not self.running_continuously:
            self.step_forward_button.config(state="normal") 
            return

        if self.current_step_index < len(self.history) - 1:
            self.current_step_index += 1
            state = self.history[self.current_step_index]
            self._update_ui_from_state(state)
        else: 
            self._toggle_continuous_run()
            self._handle_algorithm_end()
            self.step_forward_button.config(state="disabled")
            return

        if self.current_step_index == len(self.history) - 1:
            self._toggle_continuous_run()
            self._handle_algorithm_end()
            self.step_forward_button.config(state="disabled")
            return
        
        delay = int(2100 - self.animation_speed_scale.get())
        self.after(delay, self._run_loop)

    def _toggle_sidebar(self):
        current_pos = self.root_pane.sashpos(0)
        if current_pos > 0:
            self.sidebar_width = current_pos
            self.root_pane.sashpos(0, 0)
            self.toggle_sidebar_button.config(text=">>>")
        else:
            self.root_pane.sashpos(0, self.sidebar_width)
            self.toggle_sidebar_button.config(text="<<<")

    def _update_ui_from_state(self, state):
        self._update_canvas(state)
        self._update_sidebar(state)
        log_msg = state.get('log_message')
        if log_msg:
            self._log(log_msg)

    def _populate_sidebar_initial(self):
        if not self.sidebar_tree: return
        self.sidebar_tree.delete(*self.sidebar_tree.get_children())
        sorted_vertices = sorted(list(self.graph.vertices))
        for v in sorted_vertices:
            self.sidebar_tree.insert("", "end", iid=v, values=(v, "No", "∞", "-", ""))

    def _update_sidebar(self, state):
        if not self.sidebar_tree: return
        distances = state.get('distances', {})
        prev_nodes = state.get('previous_nodes', {})
        visited = state.get('visited', set())
        pq_nodes = state.get('priority_queue_nodes', [])
        current = state.get('current_vertex')
        self.sidebar_tree.delete(*self.sidebar_tree.get_children())
        sorted_vertices = sorted(list(self.graph.vertices))
        for v in sorted_vertices:
            known = "Yes" if v in visited else "No"
            cost_val = distances.get(v, float('inf'))
            cost = f"{cost_val:.1f}" if cost_val != float('inf') else "∞"
            path = self._reconstruct_path(prev_nodes, v)
            note = ""
            if v == current: note = "Current"
            elif v in pq_nodes: note = "In PQ"
            self.sidebar_tree.insert("", "end", iid=v, values=(v, known, cost, path, note))

    def _reconstruct_path(self, previous_nodes, end_vertex):
        path = []
        current = end_vertex
        if previous_nodes.get(current) is None and current not in [k for k,v in previous_nodes.items() if v is None]:
            return "-"
        while current is not None:
            path.insert(0, current)
            if len(path) > len(self.graph.vertices): return "Lỗi lặp"
            if current not in previous_nodes: break
            current = previous_nodes.get(current)
        return "->".join(path)

    def _on_node_press(self, event):
        canvas_x = self.canvas.canvasx(event.x)
        canvas_y = self.canvas.canvasy(event.y)
        item_tuple = self.canvas.find_closest(canvas_x, canvas_y)
        if not item_tuple: return
        item = item_tuple[0]
        tags = self.canvas.gettags(item)
        node_name = None
        if "node" in tags or "node_text" in tags:
            node_name = tags[1]
        if node_name:
            self._drag_data["node_name"] = node_name
            self.canvas.config(cursor="fleur")

    def _on_node_motion(self, event):
        node_name = self._drag_data.get("node_name")
        if not node_name: return
        new_x = self.canvas.canvasx(event.x)
        new_y = self.canvas.canvasy(event.y)
        self.node_display_positions[node_name] = (new_x, new_y)
        node_obj = self.node_objects[node_name]
        old_coords = self.canvas.coords(node_obj['oval'])
        old_center_x = (old_coords[0] + old_coords[2]) / 2
        old_center_y = (old_coords[1] + old_coords[3]) / 2
        dx = new_x - old_center_x
        dy = new_y - old_center_y
        self.canvas.move(node_obj['oval'], dx, dy)

        text_x, text_y, anchor = self._get_text_position_and_anchor(node_name)
        self.canvas.coords(node_obj['text'], text_x, text_y)
        self.canvas.itemconfig(node_obj['text'], anchor=anchor)

        for neighbor, weight in self.graph.get_neighbors(node_name):
            if neighbor not in self.node_display_positions: continue
            neighbor_pos = self.node_display_positions[neighbor]
            nx, ny = neighbor_pos
            key = tuple(sorted((node_name, neighbor)))
            if key in self.edge_objects:
                edge_obj = self.edge_objects[key]
                self.canvas.coords(edge_obj['line'], new_x, new_y, nx, ny)
                self.canvas.coords(edge_obj['weight'], (new_x + nx) / 2, (new_y + ny) / 2 - 8)

    def _on_node_release(self, event):
        self._drag_data["node_name"] = None
        self.canvas.config(cursor="")
        self._update_canvas_scrollregion()

In [10]:
if __name__ == "__main__":
    current_dir = os.getcwd()
    # Ghép đường dẫn lên 1 cấp, rồi vào thư mục data
    file_path = os.path.join(current_dir, '..', 'data', 'graph.csv')
    file_path = os.path.abspath(file_path)
    g = Graph()
    is_load_successful = g.load_from_csv(file_path)
    if is_load_successful:
        if not g.vertices:
            root = tk.Tk()
            root.withdraw() 
            messagebox.showerror("Lỗi dữ liệu", f"File '{file_path}' hợp lệ nhưng không chứa đỉnh nào.")
            root.destroy()
            print("\nKhông thể khởi tạo giao diện do đồ thị rỗng.")
        else:
            app = DijkstraVisualizer(g)
            app.mainloop()
    else:
        root = tk.Tk()
        root.withdraw() 
        messagebox.showerror("Lỗi nạp file", 
                             f"Không thể nạp dữ liệu từ '{file_path}'.\n"
                             f"Vui lòng kiểm tra lại file và các thông báo lỗi trong console.")
        root.destroy()
        print("\nKhông thể khởi tạo giao diện do có lỗi khi nạp dữ liệu đồ thị.")

Đã nạp 17 cạnh từ file 'd:\levy\Nam 3\DV_PTDL_ML\Lập trình phân tích dữ liệu\Dijkstra\final-ltptdl\Dijkstra\data\graph.csv'.


In [11]:
'''
# --- PHẦN THỰC THI CHÍNH ---

# 1. Khởi tạo và nạp dữ liệu
g = Graph()
try:
    g.load_from_csv("Graph.csv") # Đổi tên file nếu cần
except Exception as e:
    print(e)

print("-" * 30)

# 2. Hiển thị đồ thị
g.show_adj_list()

print("-" * 30)

# 3. Tìm đường đi ngắn nhất
start = "California"
end = "Michigan"

path, distance = g.dijkstra(start, end)

# 4. In kết quả
if path:
    print(f"Đường đi ngắn nhất từ '{start}' đến '{end}':")
    print(" -> ".join(path))
    print(f"Tổng độ dài: {distance}")
else:
    print(f"Không tìm thấy đường đi từ '{start}' đến '{end}'.")

print("-" * 30)

# Ví dụ khác
start = "Montana"
end = "Houston"
path, distance = g.dijkstra(start, end)
if path:
    print(f"Đường đi ngắn nhất từ '{start}' đến '{end}':")
    print(" -> ".join(path))
    print(f"Tổng độ dài: {distance}")
else:
    print(f"Không tìm thấy đường đi từ '{start}' đến '{end}'.")'''

'\n# --- PHẦN THỰC THI CHÍNH ---\n\n# 1. Khởi tạo và nạp dữ liệu\ng = Graph()\ntry:\n    g.load_from_csv("Graph.csv") # Đổi tên file nếu cần\nexcept Exception as e:\n    print(e)\n\nprint("-" * 30)\n\n# 2. Hiển thị đồ thị\ng.show_adj_list()\n\nprint("-" * 30)\n\n# 3. Tìm đường đi ngắn nhất\nstart = "California"\nend = "Michigan"\n\npath, distance = g.dijkstra(start, end)\n\n# 4. In kết quả\nif path:\n    print(f"Đường đi ngắn nhất từ \'{start}\' đến \'{end}\':")\n    print(" -> ".join(path))\n    print(f"Tổng độ dài: {distance}")\nelse:\n    print(f"Không tìm thấy đường đi từ \'{start}\' đến \'{end}\'.")\n\nprint("-" * 30)\n\n# Ví dụ khác\nstart = "Montana"\nend = "Houston"\npath, distance = g.dijkstra(start, end)\nif path:\n    print(f"Đường đi ngắn nhất từ \'{start}\' đến \'{end}\':")\n    print(" -> ".join(path))\n    print(f"Tổng độ dài: {distance}")\nelse:\n    print(f"Không tìm thấy đường đi từ \'{start}\' đến \'{end}\'.")'